In [9]:
# import libaries
import numpy as np
import pandas as pd
import dask.dataframe as dd
import dask.array as da
import dask.bag as db

In [10]:
path = "/data/dataprocessing/interproscan/all_bacilli.tsv"
# read tsv file
df = dd.read_csv(path, sep="\t")
# show the first 5 rows
df.head(5)


FileNotFoundError: [Errno 2] No such file or directory: '/data/dataprocessing/interproscan/all_bacilli.tsv'

22/06/24 23:08:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


AnalysisException: Path does not exist: file:/data/dataprocessing/interproscan/all_bacilli.tsv